# Homework 3: Flash/ No Flash Photography

In this homework, you will implement a subset of the idea presented in the paper 'Flash/No Flash Photography' [1]. The goal of this homework is to fuse together images captured with and without a flash.

We assume that the scene is very dim. Hence, the image without the flash will have the lighting and color characteristics you want, but it will be noisy. The image with the flash will have incorrect lighting (i.e., the colors are off), but it will be much less noisy. You will write a program to denoise the no-flash image and then transfer detail from the flash image. The original paper proposes methods to compensate for highlights and shadows caused by the flash as well as color corrections, but you will not need to implement this.

[1] Digital photography with flash and no-flash image pairs, Georg Petschnigg, Maneesh Agrawala, Hugues Hoppe. Richard Szeliski, Michael Cohen, Kentaro Toyama, SIGGRAPH 2004.


![](images/example_large.png)
<center>Figure 1: An example flash (left) and no flash (right) image pair. The flash (left) image has low noise, but the color and lighting is wrong.The colors and lighting in the no flash (right) image are what we want, but the right image is noisy. </center>

![](images/left_crop.png) ![](images/right_crop.png)
<center>Figure 2: Closeups from Fig. 1. The left (flash) image has low noise compared to the no flash (right) image.</center>

## <span style="color:red">Coding Tasks: </span>
1. Implement all functions in src.code.py

## <span style="color:blue">Writing Tasks: </span>

We have prepared a list of question that you should include at the bottom of the notebook

# <span style="color:orange">Problem 1: Load the images and explore data </span>

First, you should try to load in the images and put them into an array. When you load the images, be aware of the size of the images; it might be worth changing to a smaller resolution to work with – not too small, but still not too large as it makes the bilateral filter code run quite slowly. You can always start with something smaller and then samply change it to a larger size once you're algorithms are working smoothly!

For the example set with the "carpet"-dataset, this is not needed since it's already small enough.

In [ ]:
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
import numpy as np
import cv2
from matplotlib.pyplot import figure
import matplotlib.patches as patches

import matplotlib

In [ ]:
# Information on autoreload: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
from src import code

In [ ]:
img_noisy,img_flash = code.load_imgs('data//carpet')
print(img_flash.shape) # Should be (NumX, NumY, 3)
print(img_flash.dtype) # float32
print(img_noisy.shape) # Should be (NumX, NumY, 3)
print(img_noisy.dtype) # float32
print(img_flash.max()) # Should be close to 1 or exactly 1, but not larger
print(img_noisy.max()) # Should be close to 1 or exactly 1, but not larger

In [ ]:
#these plot the first images by themselves to see:
# If you use cv2 to read the image, make sure these images don't appear blue because of the BGR color scheme!!!
code.plot_imgs(img_noisy,img_flash)
code.save_fig_as_png("loaded_images")

# <span style="color:orange">Problem 2: Apply bilateral filer and explore </span>

A small blurb on bilateral filtering:

Bilateral filtering is an edge-detecting, noise-reducing, smoothing filter for images. It replaces the pixels with an average of the nearby pixels, which is dependent on a Gaussian kernel in the spatial domain ($σ_s$) and also the range (intensity of pixels) domain ($σ_r$)

read more at:
1. https://en.wikipedia.org/wiki/Bilateral_filter
2. https://people.csail.mit.edu/sparis/bf_course/slides/03_definition_bf.pdf
3. http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MANDUCHI1/Bilateral_Filtering.html

You can e.g. use the bilateral-filter function from cv2, but there are other implementations available too. Any  bilateral filter function will take (at least) two parameters:
1. the std deviation of the Gaussian kernel in the spatial domain $\sigma_s$
2. the std deviation of the Gaussian kernel in the range (intensity) domain $\sigma_r$

You will need to play around with these parameters to try to find the most visually pleasing results. Here are some guidelines:

1. Denoise each color channel separately.
2. Try a set of different values for both $\sigma_s$ and $\sigma_r$.

A good place to start is the range $\sigma_r \in [.05,.25]$ , and $\sigma_s \in [1,64]$ . Note that these values might assume that the maximum value in the image is 1. If the image has a different maximum value, you will need to scale by this factor. 

![](images/overview_parameter.png)
<center>Figure 3: A grid of closeup images of the no flash image from Figs. 1 and 2. The images have been denoised using the bilateral filter with varying values for the input parameters.</center>

![](images/bilateral_denoised.jpg)
<center>Figure 4: The denoised result of the no flash image from Figs. 1 and 2.</center>

In [ ]:
img_noisy.shape

In [ ]:
# Apply the bilateral filter (which you will have to implement)

In [ ]:
test_img = code.bilateral_filter(img_noisy, sigma_r=0.3, sigma_s=20)

In [ ]:
# Visualize that the bilateral filter is working
# Your plot functions should show the complete image, a cropped version and it should be indicated where you have cropped
# with a rectangle over the image (This needs to be done programmatically)
# x is in the horizontal direction, y is in the vertical direction
code.imshow_single_bilateral_filter(img_noisy,test_img,ymin=300,ymax=400,xmin=400,xmax=500)
code.save_fig_as_png("bilateral_example")

In [ ]:
# Implement this function which returns reasonable values for the paramters of the bilateral filter
range_of_sigma_r,range_of_sigma_s = code.get_range_bilateral_filter()

# This snippet will help you visualize the values that you sample for the bilateral filter
[X,Y] = np.meshgrid(range_of_sigma_r,range_of_sigma_s)
num_images = range_of_sigma_r.size * range_of_sigma_s.size
print(num_images)
plt.scatter(X,Y)
plt.xlabel("Sigma Range")
plt.ylabel("Sigma Spatial")
plt.title("Distribution of Bilateral FIlter Parameters")

code.save_fig_as_png("chosen_bilaterasl_parameters")

In [ ]:
# Let's just print them out, so that we can keep them in mind
print(range_of_sigma_s)
print(range_of_sigma_r)

In [ ]:
# This code might run for a quite while. Give it a few minutes if you process a large range of values
# Once you've run it, you can store the images in memory so you don't have to do it again
#
# NOTE: I recommend testing this function not with all the values.
#       E.g. you can pass only range_of_sigma_r[0:2] and range_of_sigma_s[0:1] and it should work too
# 
filtered_images = code.filter_all_images_with_bilateral(img_noisy, range_of_sigma_r, range_of_sigma_s)

In [ ]:
print(filtered_images.shape) # could e.g. be (Num_x, Num_y, 3 (RGB), len(range_of_sigma_r),len(range_of_sigma_r))
print(filtered_images.dtype) # Should be float32

In [ ]:
# Now you have to write a function that is visualizing the bilateral filter

In [ ]:
code.plot_bilateral_filter(filtered_images, range_of_sigma_r, range_of_sigma_s, 
                      yMin = 400, yMax = 500, xMin = 300, xMax = 400)
plt.tight_layout()

code.save_fig_as_png("bilateral_sweep_cropped")

In [ ]:
# Let's also look at the images for the full size images

In [ ]:
code.plot_bilateral_filter(filtered_images, range_of_sigma_r, range_of_sigma_s)
code.save_fig_as_png("bilateral_sweep_full")

# <span style="color:orange">Problem 3: Extract the details from the flash image and fuse the images together</span>

Now we need to extract the details from the flash image. To do this, you will apply a bilateral filter to the flash image. Use the following equations where *$F$* is flash image, denoised flash image is *$F_d$*, and the denoised no flash image as *$A_d$*. 

You will transfer the detail to the fused image *$A_f$* using the following equation:

<h1><center>$A_{f} = A_{d} * \frac{F+\epsilon}{F_{d}+\epsilon}$</center></h1>

where $\epsilon$ is a small number (e.g. 0.1 or 0.2)

You will need to choose the bilateral filter settings to generate the denoised flash image. A good place to start is to use the same settings used to generate the denoised no-flash image. Play around with these parameters to see if you can fine-tune the quality of your fused image and report your results.

In [ ]:
img_flash.shape

In [ ]:
eps = 0.1

detail, filtered = code.calc_detail_layer_from_scratch(img_flash,sigma_r =0.3,sigma_s = 10,eps=eps)

code.visualize_detail_layer_single(img_flash,filtered,detail,eps,ymin=100,ymax=350,xmin=350,xmax=600)

code.save_fig_as_png("extraction_detail_layer")

# Explore the influence of EPS and the bilateral filter parameters

Use the functions that you have implemented above to analyze the influence and calculate several detail layers for simple comparison.

HINT: Think about which parameter $\sigma_s$ or $\sigma_r$ has a larger effect on the bilateral filter. You might want to fix one value meaningful and do a grid search over eps and $\sigma_s$.

In [ ]:
# If you don't give an extra input then display the complete image
code.visualize_detail_layer(img_flash)

code.save_fig_as_png("detail_layer_sweep_full")

In [ ]:
# However cropping and zooming into the images might actually be a bit more informative
code.visualize_detail_layer(img_flash,ymin=100,ymax=350,xmin=350,xmax=600)

code.save_fig_as_png("detail_layer_sweep_crop")

## <span style="color:orange">Problem 4: Fuse the image</span>

Now we will fuse the images. This is a very simple operation. Essentially, this is just a multiplication of the detail layer with a denoised no  flash image.

You will now have to implement code.fuse_flash_no_flash.

In [ ]:
fused, detail, img_flash_filtered, img_noisy_filtered = \
    code.fuse_flash_no_flash(img_noisy,img_flash,0.1,10,0.1)

In [ ]:
code.visualize_fusion(img_flash,img_flash_filtered,img_noisy,img_noisy_filtered,
                     fused,detail,eps)
code.save_fig_as_png("fused_image_overview_full")
code.visualize_fusion(img_flash,img_flash_filtered,img_noisy,img_noisy_filtered,
                     fused,detail,eps,ymin=100,ymax=300,xmin=400,xmax=600)
code.save_fig_as_png("fused_image_overview_crop")

# <span style="color:orange">Problem 5: Write a function that does the complete pipeline</span>

In [ ]:
foldername = "cave"
sigma_r = 0.2
sigma_s = 20
eps = 0.3
img_noisy,img_flash, fused_image, detail, img_flash_filtered, img_noisy_filtered = code.complete_pipeline(foldername,sigma_r,sigma_s,eps)

In [ ]:
code.visualize_fusion(img_flash,img_flash_filtered,img_noisy,img_noisy_filtered,
                     fused_image,detail,eps)
code.save_fig_as_png("fused_image_overview_full_" + foldername)
code.visualize_fusion(img_flash,img_flash_filtered,img_noisy,img_noisy_filtered,
                     fused_image,detail,eps,ymin=100,ymax=300,xmin=400,xmax=600)
code.save_fig_as_png("fused_image_overview_crop_" + foldername)


## <span style="color:red">Coding Tasks: Calculate a fused image for each pair in the dataset</span>

There are 5 datasets, now apply this pipeline to each of the 5 folders. Report the best parameter that you've found and visualize the images in your write-up.

In [ ]:
folders = ['carpet','cave','lamp','pots','puppets']

# <span style="color:blue">Writing Tasks: </span>

### 1. Abstract
Describe in a few sentences (4-5) what the idea behind Flash-No-Flash Photography is
    - What is the problem?
    - How is it solved
   
   
### 2. Introduction
Read through the cited paper. Summarize the introduction of the paper in your own words. Your introduction should be a bit more detailed than the abstract.
 - Include a few images that motivate the problem well!
 - Try to find the gist of the paper. Keep the important stuff and remove the less important stuff.


### 3. Bilateral Filter
In this section, you will have to explain the bilateral filter in your own words!
Here are few important points:
 - Use some illustrative Figures that you find online on the bilateral filter and cite them
 - Include the essential formulas of the bilateral filter (Use latex-math code, no picture copying)
 - Explain well the effect of the two parameters $\sigma_s$ and $\sigma_r$
 - Show how the bilateral filter works for the dataset that we provided. Show example images and write a few sentences about it
 - Is the bilinear filter linear (e.g., like a Gaussian filter)? If yes, explain why. If not, explain why. 
 - What are the shortcomings of the bilateral filter? 
- I assume that you know a little bit about Deep Learning: What if you learn a CNN-style network to do a similar task (edge-preserving-filter). Do you think this could work? If yes, can you explain? If you've never heard of the bilateral filter, did you find other edge-preserving filters on google? If yes, can you tell us what the main idea is?  How are they different (note: keep this section short, don't spend too much time on this literature research)

### 4. Flash No-Flash Photography
Explain in your own words how flash - noflash photography works. Look up the paper, extract the crucial paragraphs, and summarize them in your own words.

Also, try to answer the following questions in your text:
- Will this approach work with a linear filter such as a Gaussian Filter? Try to find good reasoning why you come to this conclusion. There is no right or wrong. We want to understand your thought process.
- Where do you think the approach you have implemented will fail? Think about what (physically) can happen to the scene or to the camera that will lead to severe artifacts?
- Do you have any ideas/suggestions on how this method can be improved? 

Include the formula for the detail-transfer in your write-up. Show the results that you have obtained from your implementation to visualize how the method works. 

### 5. Results

Include the best results from your implementation from all five datasets into your report. 
 - Highlight the region where the method shines (i.e., you see clear improvement)
 - Highlight some regions where the method does not work well. Try to explain why these regions didn't work.
 
### 6. Conclusion
Summarize what you've learned in this assignment. What was hard to understand? What was hard to implement?
 